In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import numpy as np

In [ ]:
import pymorphy2
import pandas as pd
import glob
import csv

path = '/home/kirill/Загрузки/2_Master.txt'

test_txt = open(path,encoding="utf8")
#mas_texts = test_txt.read().split("stopword")
#print(mas_texts)
data = test_txt.read()
title = data.split('\n', 1)[0]
data = data.split('\n', 2)[1]
print(title)
print(data)
test_txt.close()

test_txt = open(path,encoding="utf8")
#for i in range(len(mas_texts)):
#    f = open('/home/kirill/Загрузки/New'+str(i)+'.txt','w')
#    f.write(mas_texts[i])
#    f.close()

##text = test_txt.read()
##res = len(text.split())
##print(res)

morph = pymorphy2.MorphAnalyzer()
Unused_chars=[':',',','.','-','\n','?','!',')','(', '«', '»']
POS = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'ADVB', 'NPRO']
normal_form_words = []

import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

for line in test_txt:
        l=line.split('.')
        for sentense in l:
            #print(preprocess_text(sentense))
            new_sent = preprocess_text(sentense)
            words = new_sent.split()
            for i in range(len(words)):
                #Добавление слова в список слов и избавление от лишних символов
                temp = words[i]
                for char in Unused_chars:
                    if char in temp:
                        temp=words[i].replace(char,'')
                        
                word_pos = morph.parse(temp)[0].tag.POS
                if(word_pos in POS):
                    normal_form_words.append(morph.parse(temp)[0].normal_form)

test_txt.close()

#Слова для анализа

emotions = open('/home/kirill/dictionary.csv')

df = pd.read_csv(emotions)
df = df.drop(df.columns[[0]], axis=1)

pat = '|'.join(normal_form_words)

new = df[df['Russian Translation (Google Translate)'].str.contains(pat)]
new = new.reset_index(drop = True)
print(new)    

##########################################

#Все вхождения
df.rename(columns={'Russian Translation (Google Translate)': 'Russian'}, inplace=True)  
df = df.sort_values(by = ['Russian'])

pat = '|'.join(df['Russian'])

list_of_words = []

for each in normal_form_words:
    if (each in pat):
        list_of_words.append(each)
        
list_of_words.sort()
print(list_of_words)

df_list_of_words = pd.DataFrame(pd.np.empty((0, 11)))
df_list_of_words.columns = df.columns
df_list_of_words['Russian'] = list_of_words

new_list = []

for each in df_list_of_words.itertuples():
    for row in df.itertuples():
        if (each[1] == row[1]):
            
            new_list.append(list(row[1:12]))
            
words_with_emotions = pd.DataFrame(new_list)
words_with_emotions.columns = df.columns     

##########################################

print(len(words_with_emotions))

class Labeled_Words:
    def __init__(self, words, targets, target_names):
        self.words = words
        self.targets = targets
        self.target_names = target_names
        
    def __eq__(self, other):
        if isinstance(other, Number):
            return self.number == other.number
        return False

##########################################

list_of_Labeled_words = Labeled_Words([],[],[])
list_of_Labeled_words.words = words_with_emotions['Russian']
list_of_Labeled_words.targets = words_with_emotions['Positive']

##########################################

print("Точность на обучающей выборке:\n")

docs_test = list_of_Labeled_words.words

nb_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('mnb', MultinomialNB()),
                    ])

nb_clf = nb_clf.fit(list_of_Labeled_words.words, list_of_Labeled_words.targets)

predicted = nb_clf.predict(docs_test)
nb = np.mean(predicted == list_of_Labeled_words.targets)
print("CV + TF-IDF + NB: ", nb) 

svm_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))
                    ])

svm_clf = svm_clf.fit(list_of_Labeled_words.words, list_of_Labeled_words.targets)

predicted = svm_clf.predict(docs_test)
svm = np.mean(predicted == list_of_Labeled_words.targets)
print("CV + TF-IDF + SVM: ", svm) 

knn_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('knn', KNeighborsClassifier(n_neighbors=5, p=2, algorithm='kd_tree'))
                    ])

knn_clf = knn_clf.fit(list_of_Labeled_words.words, list_of_Labeled_words.targets)

predicted = knn_clf.predict(docs_test)
knn = np.mean(predicted == list_of_Labeled_words.targets)
print("CV + TF-IDF + k-NN: ", knn) 

print("Точность на тестовой выборке:\n")

docs_test = words_with_emotions['Russian']

predicted = nb_clf.predict(docs_test)
nb = np.mean(predicted == list_of_Labeled_words.targets)
print("CV + TF-IDF + NB: ", nb) 

predicted = svm_clf.predict(docs_test)
svm = np.mean(predicted == list_of_Labeled_words.targets)
print("CV + TF-IDF + SVM: ", svm) 

predicted = knn_clf.predict(docs_test)
knn = np.mean(predicted == list_of_Labeled_words.targets)
print("CV + TF-IDF + k-NN: ", knn) 